Library for collected data.

In [1]:
import requests
import pandas as pd
import json
from bs4 import BeautifulSoup
import os
import time


Search list of Problem Detail such as contestid, index of problem by requested from CodeForce API

In [ ]:
probSet = requests.get("https://codeforces.com/api/problemset.problems").json()
nameId = pd.DataFrame(columns={'contestId','name'})
for i in probSet['result']['problems']:
    nameId = nameId.append({'contestId':i['contestId'],'name':i['name']},ignore_index=True)



We list the problem that only have both pass in Python and C++ languages, then filter out only have vecdict status accepted.

In [ ]:
listOfProblem = ['WebofLies','GregorandCryptography','PolycarpandCoins','WonderfulColoring','SubsequencePermutation','DigitsSum','RunningforGold',
'XORandDistance','Cherry','Cobb','Mikasa','AVarietyofOperations','EzzatandTwoSubsequences','MochaandMath','DislikeofThrees',
'SimplyStrangeSort','MedianMaximization','SeatingArrangementseasyversion','NonDecreasingDilemma','BalancedSubstring','Book',
'RegularBracketSequences']
conId = set(nameId[nameId.isin(lst_AlphaCode)['name'] == True]['contestId'])
probInfo = {}
for i in conId:
    print(i)
    ci = requests.get(f'https://codeforces.com/api/contest.status?contestId={i}').json()
    for j in ci['result']:
        pni = j['problem']['name']+'#'+str(i)+' '+j['problem']['index']
        if(pni.split('#')[0] in lst_AlphaCode):
            # print(pni.split('#')[0])
            if(j['verdict'] == 'OK' and ('C++' in j['programmingLanguage'] or 'Python' in j['programmingLanguage'])):
                if(pni not in probInfo):
                    probInfo[pni] = {j['id']:{'language':j['programmingLanguage'],'handle': j['author']['members'][0]['handle']}}
                else:
                    probInfo[pni][j['id']] = {'language':j['programmingLanguage'],'handle': j['author']['members'][0]['handle']}




This part is used to scape human code submissions from the CodeForce website, then filter only the code parts on the website and save the file to the directory. We implement this code for collecting sample selected problems from codeforce. The limitation is 1,000 human codes in both Python and C++ languages.

In [6]:
def writeFile(main_dir,url,lang,typ,iD,nameInfo):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    cf = [ f.text for f in soup.find_all("pre",class_="prettyprint lang-py linenums program-source") if(len(f.text) >1)]
    complete_name = os.path.join(main_dir,lang+'/'+nameInfo+'/'+ str(iD)+typ)
    try:
        file = open(complete_name,"w")
        file.write(cf[0])
        return True
    except:
        while(True):
            print('Waiting for 3mins to request again')
            time.sleep(310)
            page = requests.get(url)
            soup = BeautifulSoup(page.content, 'html.parser')
            cf = [ f.text for f in soup.find_all("pre",class_="prettyprint lang-py linenums program-source") if(len(f.text) >1)]
            try:
                file = open(complete_name,"w")
                file.write(cf[0])
                return True
            except:
                print('\nRecon----->\n',complete_name) 
        return False
#Main 
lang = ["Python",'C++']
delay=0
for i in probInfo.keys():
    cntC,cntPy = 1,1
    if(i.split('#')[0] in lst_AlphaCode):
        print(i)
        conInfo = i.split('#')[1].split(' ')
        nameInfo = i.split('#')[0]
        main_dir = '/Users/silalertbanjongngam/Desktop/AlphaCode/'
        num_file =[]
        for l in lang:
            tmp_dir = main_dir + l + '/' 
            path = os.path.join(tmp_dir, nameInfo)
            print(path)
            try:
                os.mkdir(path)
            except:
                print('Error----->',path)
        for j in probInfo[i]:
            if(f'{j}.py' in os.listdir(f'/Users/silalertbanjongngam/Desktop/AlphaCode/Python/{nameInfo}')):
                continue
            url = f'https://codeforces.com/contest/{conInfo[0]}/submission/{j}'
            if('Python' in probInfo[i][j]['language'] and cntPy <1001):
                try:
                    writeFile(main_dir,url,'Python','.py',str(j),nameInfo)
                except:
                    while(True):
                        if( writeFile(main_dir,url,'Python','.py',str(j),nameInfo)): 
                            break
                cntPy+=1
                if(cntPy%100 ==0):
                    print('py -> ',cntPy)
            if('C++' in probInfo[i][j]['language'] and cntC < 1001 ):
                try:
                    writeFile(main_dir,url,'C++','.cpp',str(j),nameInfo)
                except:
                    while(True):
                        if( writeFile(main_dir,url,'C++','.cpp',str(j),nameInfo)):
                            break
                cntC+=1
                if(cntC%100 ==0):
                    print('c++ -> ',cntC)
            if((cntPy%100 == 0)):
                print('Delay for 400SEC')
                time.sleep(400)
                delay+=1
                cntPy+=1
                if(delay>=3):
                    break
            else:
                delay=0

Web of Lies#1548 A
/Users/silalertbanjongngam/Desktop/AlphaCode/Python/Web of Lies
Error-----> /Users/silalertbanjongngam/Desktop/AlphaCode/Python/Web of Lies
Gregor and the Two Painters#1548 E
/Users/silalertbanjongngam/Desktop/AlphaCode/Python/Gregor and the Two Painters
Error-----> /Users/silalertbanjongngam/Desktop/AlphaCode/Python/Gregor and the Two Painters
The Three Little Pigs#1548 C
/Users/silalertbanjongngam/Desktop/AlphaCode/Python/The Three Little Pigs
Error-----> /Users/silalertbanjongngam/Desktop/AlphaCode/Python/The Three Little Pigs
Gregor and Cryptography#1549 A
/Users/silalertbanjongngam/Desktop/AlphaCode/Python/Gregor and Cryptography
Error-----> /Users/silalertbanjongngam/Desktop/AlphaCode/Python/Gregor and Cryptography
Web of Lies#1549 C
/Users/silalertbanjongngam/Desktop/AlphaCode/Python/Web of Lies
Error-----> /Users/silalertbanjongngam/Desktop/AlphaCode/Python/Web of Lies
The Three Little Pigs#1549 E
/Users/silalertbanjongngam/Desktop/AlphaCode/Python/The Three 